In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import os

class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)
        padded_img = F.pad(img_tensor, (0, 4, 0, 4))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

transform = transforms.Compose([
    FFTTransform(),
])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_images = trainset.data.unsqueeze(1)
test_images = testset.data.unsqueeze(1)

torch.save({
    'train_images': train_images,
    'train_labels': trainset.targets,
    'test_images': test_images,
    'test_labels': testset.targets,
}, './transformed_mnist.pt')

print("Dataset saved to disk.")


CIFR10

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import os

class FFTTransform:
    def __call__(self, img):
        img_tensor = transforms.functional.to_tensor(img)
        padded_img = F.pad(img_tensor, (0, 2, 0, 2))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

transform = transforms.Compose([
    FFTTransform(),
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './transformed_cifar10.pt')

print("Dataset saved to disk.")

In [ ]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os

class HSVTransform:
    def __call__(self, img):
        img = img.convert("HSV")  # Converte l'immagine in HSV
        img_tensor = TF.to_tensor(img)  # Converte in tensore PyTorch
        padded_img = F.pad(img_tensor, (0, 2, 0, 2))
        fft_result = torch.fft.fft2(padded_img)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

# Definiamo la trasformazione
transform = transforms.Compose([
    HSVTransform(),
])

# Carichiamo il dataset con la nuova trasformazione
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Convertiamo le immagini in tensori
train_images = torch.stack([img for img, _ in trainset])
test_images = torch.stack([img for img, _ in testset])

# Salviamo il dataset trasformato
torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor(trainset.targets),
    'test_images': test_images,
    'test_labels': torch.tensor(testset.targets),
}, './transformed_cifar10_hsv.pt')

print("Dataset in formato HSV salvato su disco.")


In [ ]:

from fastai.data.external import untar_data, URLs

path = untar_data(URLs.IMAGEWOOF)
print(path)

In [ ]:
from fastai.vision.all import *

path = untar_data(URLs.IMAGEWOOF)
dls = ImageDataLoaders.from_folder(path, valid='val', item_tfms=Resize(224))

dls.show_batch(max_n=9, figsize=(7,8))

In [3]:
import torch
from torchvision import datasets, transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
import os
import random

class HSVTransform:
    def __call__(self, img):
        # Applichiamo il padding solo se l'immagine è più piccola di 224x224
        width, height = img.size
        padding = (0, 0, 0, 0)  # Padding: (left, top, right, bottom)

        if width < 224 or height < 224:
            padding = (max(0, (224 - width) // 2), max(0, (224 - height) // 2),
                       max(0, (224 - width + 1) // 2), max(0, (224 - height + 1) // 2))

        img = TF.pad(img, padding)  # Aggiungi il padding
        img = img.convert("HSV")  # Converte l'immagine in HSV
        img_tensor = TF.to_tensor(img)  # Converte in tensore PyTorch
        fft_result = torch.fft.fft2(img_tensor)
        fft_magnitude = torch.abs(fft_result)
        fft_magnitude = fft_magnitude / fft_magnitude.max()
        return fft_magnitude

# Definiamo la trasformazione
transform = transforms.Compose([
    HSVTransform(),  # Trasformazione HSV e FFT
    transforms.RandomCrop(224),  # Random crop a 224x224
])

# Percorso al dataset
dataset_path = '/home/lapo-chiostrini/.fastai/data/imagewoof2'

# Carichiamo il dataset con la nuova trasformazione
trainset = ImageFolder(root=os.path.join(dataset_path, 'train'), transform=transform)
testset = ImageFolder(root=os.path.join(dataset_path, 'val'), transform=transform)

# Otteniamo un quarto delle immagini
train_size = len(trainset)
test_size = len(testset)

train_indices = random.sample(range(train_size), train_size // 4)
test_indices = random.sample(range(test_size), test_size // 4)

train_subset = Subset(trainset, train_indices)
test_subset = Subset(testset, test_indices)

# Creiamo i DataLoader
trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)
testloader = DataLoader(test_subset, batch_size=64, shuffle=False)

# Convertiamo le immagini in tensori
train_images = torch.stack([img for img, _ in train_subset])
test_images = torch.stack([img for img, _ in test_subset])

# Salviamo il dataset trasformato
torch.save({
    'train_images': train_images,
    'train_labels': torch.tensor([trainset.targets[i] for i in train_indices]),
    'test_images': test_images,
    'test_labels': torch.tensor([testset.targets[i] for i in test_indices]),
}, './transformed_imagewoof_hsv_quarter_randomcrop_with_padding.pt')

print("Un quarto del dataset con random crop a 224x224, padding e trasformazione HSV salvato su disco.")


Un quarto del dataset con random crop a 224x224, padding e trasformazione HSV salvato su disco.
